# Welcome to the Lab 🥼🧪
## Which metros have the highest net acquisitions from investors? 

In this notebook, we are going to create a table of the last 6 months of the investor purchase to sale ratio for the top 50 metros. This is a ratio showing investor acquisitions/dispositions.

If investors bought and sold 10 homes in month, 10/10 = 1. No change in purchase to sale ratio. 

If investors bought 10 homes and sold 100 homes, 10/100 = .1. For every home bought, they are selling 10.

**Note** This notebook will work with any of the 70k+ markets supported by the Parcl Labs API.

As a reminder, you can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along. 

To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY` as a secret. See this [guide](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75) for more information.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-examples/blob/main/python/inspiration/table_of_purchase_to_sale_ratio.ipynb)

We will be creating this table: 

![Chart](assets/purchase_to_sale_ratio.png)

In [ ]:
# Environment setup
import os
import sys
import subprocess
from datetime import datetime

# Collab setup from one click above
if "google.colab" in sys.modules:
    from google.colab import userdata
    %pip install parcllabs plotly kaleido
    api_key = userdata.get('PARCL_LABS_API_KEY')
else:
    api_key = os.getenv('PARCL_LABS_API_KEY')

In [ ]:
import parcllabs
import pandas as pd
import plotly.express as px
from parcllabs import ParclLabsClient

print(f"Parcl Labs Version: {parcllabs.__version__}")

In [ ]:
# instantiate the client
client = ParclLabsClient(api_key=api_key)

In [ ]:
# lets get all metros in the country
metros = client.search_markets.retrieve(
    location_type='CBSA',
    as_dataframe=True,
    params={
        'limit': 1000
    }
)

In [ ]:
# get the ids
metro_ids = metros['parcl_id'].tolist()

In [ ]:
# get most recent housing stock for all metros
housing_stock = client.market_metrics_housing_stock.retrieve_many(
    parcl_ids=metro_ids,
    params={
        'limit': 1 # let's get the most recent stock
    },
    as_dataframe=True # if we want to get the data as a pandas dataframe
)

housing_stock.head()

In [ ]:
# lets select the top 50 metros by housing stock
top_50 = housing_stock.sort_values('all_properties', ascending=False).head(50)
top_50.head()

In [ ]:
# now lets hit the purchase to sale ratio for those 50 metropolitan areas
purchase_to_sale = client.investor_metrics_purchase_to_sale_ratio.retrieve_many(
    parcl_ids=top_50['parcl_id'].tolist(),
    params={
        'limit': 6, # get last 6 months
    },
    as_dataframe=True
)

purchase_to_sale.head()

In [ ]:
# merge the data with the metropolitan areas info to get the names
out = pd.merge(metros[['parcl_id', 'name']], purchase_to_sale[['parcl_id',  'date', 'purchase_to_sale_ratio']], on='parcl_id')
out = out.drop('parcl_id', axis=1)
out.head()

In [ ]:
# define a function to clean up the official names and make them easier to read
def format_names(nme):
    metro = nme.split(',')[0].split('-')[0].strip()
    metro = metro.split('/')[0].strip()
    return metro

out['name'] = out['name'].apply(lambda x: format_names(x))

In [ ]:
# we pivot the data for easy charting and sort by the highest purchase to sale ratio for the latest available month
charting_data = out.pivot_table(values='purchase_to_sale_ratio', index='name', columns='date')
charting_data = charting_data.sort_values('2024-03-01', ascending=False)
charting_data.head()

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from datetime import datetime

# Define a function to scale the color based on the value
def color_scale(value):
    # Normalize the value to the new range [0, 2] with a midpoint at 1
    if value < 1:
        # Scale from light blue to blue as value approaches 1 from 0
        normalized = value  # No need to normalize since it's already 0 to 1
        r = int(173 + (65 - 173) * normalized)  # Decrease red channel towards darker blue
        g = int(216 + (105 - 216) * normalized)  # Decrease green channel towards darker blue
        b = int(230 + (225 - 230) * normalized)  # Slightly increase blue channel towards 225
    else:
        # Scale from blue to dark blue as value goes from 1 to 2
        normalized = (value - 1) / 1  # Normalize the upper half from 0 to 1
        r = int(65 + (0 - 65) * normalized)  # Decrease red channel towards darkest blue
        g = int(105 + (0 - 105) * normalized)  # Decrease green channel towards darkest blue
        b = int(225 + (139 - 225) * normalized)  # Decrease blue channel towards darker blue

    # Ensure RGB values are within the valid range
    r = max(0, min(255, r))
    g = max(0, min(255, g))
    b = max(0, min(255, b))

    return f'rgb({r},{g},{b})'

# Define a function to format the date
def format_date(date):
    date = datetime.strptime(date, '%Y-%m-%d')
    return date.strftime('%b `' + date.strftime('%y'))

dates = charting_data.columns.tolist()

In [ ]:
# define the parameters for the visualization

# Define the dictionary that points to logo to be displayed on the chart
labs_logo_lookup = {
    'blue': 'https://parcllabs-assets.s3.amazonaws.com/powered-by-parcllabs-api.png',
    'white': 'https://parcllabs-assets.s3.amazonaws.com/powered-by-parcllabs-api-logo-white.png'
}

labs_logo_dict = dict(
    source=labs_logo_lookup['white'],
    xref="paper",
    yref="paper",
    x=0.5,
    y=1.02,  # Adjusted for more data
    sizex=0.3,
    sizey=0.2,
    xanchor="center",
    yanchor="bottom"
)

# define the table
fig = go.Figure(
    data=[go.Table(
        header=dict(
            values=['<b>Market</b>'] + [f"<b>{format_date(date)}</b>" for date in dates],
            fill_color='#080D16',
            font=dict(color='#FFFFFF', size=12),
            align='center',
            height=30
        ),
        cells=dict(
            values=[[f"{name}" for name in charting_data.index]] + 
                   [charting_data[date] for date in dates],
            fill=dict(
                color=[
                    ['#080D16']*len(charting_data.index)] +
                    [[color_scale(charting_data.at[market, date]) for market in charting_data.index] for date in dates]
            ),
            font=dict(
                color='#FFFFFF',
                size=11  # Uniform size for all cells
            ),
            align='center',
            height=30
        ))
])

fig.add_layout_image(labs_logo_dict)

fig.update_layout(
    title={
        'text': 'Investor Purchase to Sale Ratio Top 50 Metro Markets by Housing Stock',
        'y': 0.98,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    title_font_color='#FFFFFF',
    width=900,  # Slightly wider to accommodate more columns
    height=1600, # + 100 * len(charting_data.index),  # Adjust height based on the number of markets
    paper_bgcolor='#080D16',
    margin=dict(l=10, r=10, t=120, b=10)
)

fig.show()